# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Nicolas Herrera
- Nombre de alumno 2: Lucas Carrasco

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/vspartamo/MDS7202)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [1]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

ERROR: Could not build wheels for box2d-py, which is required to install pyproject.toml-based projects


### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [2]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

`escriba su respuesta acá`

#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [3]:
import gymnasium as gym
env = gym.make("LunarLander-v2", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

DependencyNotInstalled: Box2D is not installed, run `pip install gymnasium[box2d]`

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [ ]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

`escriba su respuesta acá`

#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [4]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001981585DDC0>, default_metadata=())

### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [6]:
%pip install --upgrade --quiet PyPDF2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import PyPDF2

doc_paths = ['AttentionIsAllYouNeed.pdf', 'BERT.pdf', 'ChainOfThought.pdf'] # rellenar con los path a sus documentos

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

In [8]:
from langchain_community.document_loaders import PyPDFLoader

docs = []
for path in doc_paths:
    loader = PyPDFLoader(path)
    docs += loader.load()
    
docs[:5]

[Document(metadata={'source': 'AttentionIsAllYouNeed.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architectu

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)
splits[:5]

[Document(metadata={'source': 'AttentionIsAllYouNeed.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu'),
 Document(metadata={'source': 'AttentionIsAllYouNeed.pdf', 'page': 0}, page_content='University of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best

In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)
vectorstore

#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

In [11]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever_chain = retriever | format_docs

#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# noten como ahora existe el parámetro de context!
rag_template = '''
Eres un asistente experto en investigación sobre LLMs.
Tu único rol es contestar preguntas del usuario a partir de información relevante que te sea proporcionada.
Responde siempre de la forma más completa posible y usando toda la información entregada.
Responde sólo lo que te pregunten a partir de la información relevante, NUNCA inventes una respuesta.

Información relevante: {context}
Pregunta: {question}
Respuesta:
'''

rag_prompt = PromptTemplate.from_template(rag_template)

rag_chain = (
    {
        "context": retriever_chain,
        "question": RunnablePassthrough(),
    } # Los parámetros de la plantilla
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [21]:
question_answer_list = [
    (
        "Qué es un Transformer?",
        "Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales."
    ),
    (
        "Según Chain of Thought, ¿qué técnica se puede usar para resolver un problema complejo con un LLM?",
        "Se puede descomponer el problema en partes más pequeñas y resolver cada parte por separado."
    )
]

for question_answer in question_answer_list:
    res = rag_chain.invoke(question_answer[0])
    print("Respuesta esperada: ", question_answer[1])
    print("Respuesta obtenita: ", res)
    print()

Respuesta esperada:  Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales.
Respuesta obtenita:  Un Transformer es un modelo de transducción que utiliza únicamente auto-atención para calcular representaciones de su entrada y salida, sin usar RNNs alineados con la secuencia o convoluciones.  Su arquitectura se compone de un codificador y un decodificador, ambos formados por una pila de N=6 capas idénticas. Cada capa tiene dos subcapas: la primera es un mecanismo de auto-atención multi-cabeza, y la segunda es una capa de alimentación directa totalmente conectada.  Tanto el codificador como el decodificador utilizan codificaciones posicionales.  El modelo base utiliza una tasa de abandono (dropout) de Pdrop = 0.1 y durante el entrenamiento se emplea suavizado de etiquetas con un valor de ϵls = 0.1.


Respuesta esperada:  Se puede descomponer el problema en partes más pequeñas y resolver cada parte por separado.
R

**RESPUESTA**: La solución de RAG entrega respuestas correctas, y bastante parecidas a las esperadas. Al ser preguntas no simples, era difícil que la respuesta fuera exactamente igual, pero en general, la respuesta entregada por RAG es correcta.

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

In [22]:
def try_other_config(
    chunk_size: int = 500, 
    ammount_of_chunks: int = 3, 
    search_type: str = 'similarity'
):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=50)
    splits = text_splitter.split_documents(docs)

    embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
    vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento

    retriever = vectorstore.as_retriever(
        search_type=search_type, # método de búsqueda
        search_kwargs={"k": ammount_of_chunks}, # n° documentos a recuperar
    )

    retriever_chain = retriever | format_docs

    rag_chain = (
        {
            "context": retriever_chain,
            "question": RunnablePassthrough(),
        }
        | rag_prompt
        | llm
        | StrOutputParser()
    )

    for question_answer in question_answer_list:
        res = rag_chain.invoke(question_answer[0])
        print("Respuesta esperada: ", question_answer[1])
        print("Respuesta obtenita: ", res)
        print()


**PROBANDO CAMBIAR TAMAÑO DE CHUNK**

In [23]:
try_other_config(chunk_size=50)

Respuesta esperada:  Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales.
Respuesta obtenita:  Basado en la información proporcionada, un Transformer es un modelo que puede ser usado para tareas de traducción.  También se menciona que existe una versión del Transformer referida como "Transformer".


Respuesta esperada:  Se puede descomponer el problema en partes más pequeñas y resolver cada parte por separado.
Respuesta obtenita:  Según la información proporcionada, la técnica "chain of thought" es particularmente adecuada para resolver problemas complejos con un LLM.  El ejemplo dado muestra que un modelo que produce un "chain of thought" puede resolver un problema.  Sin embargo, la información no describe la técnica en sí misma, solo indica su idoneidad para este propósito.




**RESPUESTA**: Al achicar el tamaño de los chunks se obtienen respuestas especificas pero distintas a lo que buscabamos. La respuesta es incompleta y difiere de lo que se esperaba. Esto se debe a que al achicar el tamaño de los chunks, se obtiene menos información y contexto para responder la pregunta.

In [24]:
try_other_config(chunk_size=750)

Respuesta esperada:  Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales.
Respuesta obtenita:  Un Transformer es una nueva arquitectura de red simple basada únicamente en mecanismos de atención, que prescinde por completo de la recurrencia y las convoluciones.  Está compuesta por un codificador y un decodificador, ambos con pilas de capas de auto-atención multi-cabeza y redes completamente conectadas punto a punto.  El modelo codificador tiene una pila de N=6 capas idénticas, cada una con dos subcapas: una de auto-atención multi-cabeza y otra red de avance completamente conectada.  Se utiliza una conexión residual alrededor de cada subcapa, seguida de normalización por capas.  Se aplica abandono (dropout) a la salida de cada subcapa, antes de sumarla a la entrada de la subcapa y normalizarla.  También se aplica abandono a la suma de las incrustaciones y las codificaciones posicionales.


Respuesta esperada: 

**RESPUESTA**: Al aumentar el tamaño de los chunks se obtiene información muchisimo más completa. Al tener tanta información, y dejar que el LLM la procese, se obtiene una respuesta mucho más completa y correcta.

**PROBANDO CAMBIAR CANTIDAD DE CHUNKS**

In [25]:
try_other_config(ammount_of_chunks=1)

Respuesta esperada:  Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales.
Respuesta obtenita:  Basado en la información proporcionada, un Transformer es un modelo de arquitectura que utiliza capas apiladas de auto-atención y capas totalmente conectadas puntuales, tanto para el codificador como para el decodificador.  El codificador está compuesto por una pila de N=6 capas idénticas, cada una con dos subcapas: una de mecanismo de auto-atención multi-cabeza y otra, una capa totalmente conectada simple y posicional.  La figura 1 muestra la arquitectura completa del modelo, con el codificador en la mitad izquierda y el decodificador en la mitad derecha.


Respuesta esperada:  Se puede descomponer el problema en partes más pequeñas y resolver cada parte por separado.
Respuesta obtenita:  Según la información proporcionada, Chain of Thought es una técnica que se puede usar para resolver problemas complejos con un 

**RESPUESTA**: Al disminuir la cantidad de chunks recuperados, se obtiene una respuesta más general y menos precisa, ya que el modelo obtieen menos contexto de diferentes partes del documento. Sin embargo, la respuesta en este caso es bastante buena, probablemente debido a que el tamaño de los chunks es lo suficientemente grande para que el modelo pueda responder correctamente, y que el chunk que se obtiene es realmente suficiente.

In [26]:
try_other_config(ammount_of_chunks=8)

Respuesta esperada:  Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales.
Respuesta obtenita:  El Transformer es una nueva y simple arquitectura de red propuesta como modelo de transducción de secuencias.  A diferencia de los modelos dominantes basados en redes neuronales recurrentes o convolucionales complejas, el Transformer utiliza únicamente auto-atención para calcular representaciones de su entrada y salida, sin usar RNNs o convoluciones alineados con la secuencia.  Es el primer modelo de transducción que hace esto.  Su arquitectura consta de un codificador y un decodificador, ambos compuestos por una pila de N=6 capas idénticas. Cada capa tiene dos subcapas: un mecanismo de auto-atención multi-cabeza y una capa totalmente conectada puntual.  El modelo utiliza codificaciones posicionales tanto en el codificador como en el decodificador, y durante el entrenamiento se emplea suavizado de etiquetas con un 

**RESPUESTA**: Al aumentar la cantidad de chunks recuperados, se obtiene una respuesta más completa y precisa, ya que el modelo obtiene más contexto de diferentes partes del documento. Esto se cumple en este caso, entregando una respuesta bastante completa.

**PROBANDO CAMBIAR TIPO DE BUSQUEDA**

In [27]:
try_other_config(search_type='mmr')

Respuesta esperada:  Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales.
Respuesta obtenita:  Un Transformer es un modelo de arquitectura que utiliza capas apiladas de auto-atención y capas totalmente conectadas puntuales, tanto para el codificador como para el decodificador.  El codificador está compuesto por una pila de N=6 capas idénticas, cada una con dos subcapas: un mecanismo de auto-atención multi-cabeza y una capa de conexión totalmente conectada posicional.  Se han desarrollado Transformers con diferentes tamaños; por ejemplo, Vaswani et al. (2017) creó uno con (L=6, H=1024, A=16) y 100M parámetros, mientras que el más grande encontrado en la literatura (Al-Rfou et al., 2018) tiene (L=64, H=512, A=2) y 235M parámetros.  Estos tamaños se comparan con BERT BASE (110M parámetros) y BERT LARGE (340M parámetros).  Un Transformer de 4 capas logró un F1 de 91.3 en el análisis sintáctico de la constitución

**RESPUESTA**: Al cambiar similarity por MMR (Maximal Marginal Relevance), se obtiene una respuesta más completa gracias a la diversidad de los chunks recuperados. Esto dado a que MMR busca chunks que sean diferentes entre sí, lo que permite que el modelo tenga más contexto para responder la pregunta.

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

In [29]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results = 1)

#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

In [30]:
%pip install wikipedia

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(lang='es', top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

In [32]:
from langchain import hub

react_prompt = hub.pull("hwchase17/react") # template de ReAct
print(react_prompt.template)

c:\Users\lucas\python-envs\nb-env\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
from langchain.agents import create_react_agent, AgentExecutor

tools = [tavily_tool, wiki_tool]

agent = create_react_agent(llm, tools, react_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', 'tool_names': 'tavily_search_results_json, wikipedia'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\

In [34]:
def call_agent(input: str):
    response = agent_executor.invoke({"input": input})
    print(response["output"])

call_agent("Dame el nombre de algunos profesores de la FCFM de la Universidad de Chile")



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find a list of professors from the Faculty of Physics and Mathematics (FCFM) at the University of Chile.  A Wikipedia search might be a good starting point, but it might not be completely up-to-date.

Action: wikipedia
Action Input: "Faculty of Physics and Mathematics, University of Chile"
Page: Universidad de California en Berkeley
Summary: La Universidad de California en Berkeley (en idThought: That was the wrong Wikipedia page. I need to try a different search term.  Let's try searching directly for the faculty's website.  If that doesn't work, I'll try a different approach.

Action: wikipedia
Action Input: "Facultad de Ciencias Físicas y Matemáticas, Universidad de Chile"
Page: Facultad de Ciencias Físicas y Matemáticas de la Universidad de Chile
Summary: La Facultad de Thought: The Wikipedia page for the Facultad de Ciencias Físicas y Matemáticas (FCFM) of the University of Chile should list facult

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

In [73]:
call_agent("¿A qué se dedica Tailor Swift?")



> Entering new AgentExecutor chain...
Thought: To answer what Taylor Swift does, I need to find information about her profession.  Wikipedia is a good resource for this.

Action: wikipedia
Action Input: Taylor Swift
Page: Taylor Swift
Summary: Taylor Alison Swift (West Reading, Pensilvania; 13 de diciembre de 1989)Thought: The Wikipedia page gives biographical information, but I need to extract her profession.  The summary doesn't explicitly state it, so I'll look further into the page content.

Action: wikipedia
Action Input: Taylor Swift
Page: Taylor Swift
Summary: Taylor Alison Swift (West Reading, Pensilvania; 13 de diciembre de 1989)Thought: The Wikipedia page should contain information about her career. I'll look for sections like "Career" or "Profession".

Action: wikipedia
Action Input: Taylor Swift
Page: Taylor Swift
Summary: Taylor Alison Swift (West Reading, Pensilvania; 13 de diciembre de 1989)Thought: I need to actually *read* the Wikipedia page content this time, not ju

In [62]:
call_agent("¿Quién fue el último fichaje del Real Madrid?")



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find information about the most recent player signing for Real Madrid.  A quick search on Wikipedia might suffice, but a more up-to-date source like a search engine might be better to ensure I have the most recent information.

Action: tavily_search_results_json
Action Input: "Real Madrid latest signing"
[{'url': 'https://en.as.com/soccer/possible-mbappe-signing-with-real-madrid-live-updates-online-as-french-striker-set-to-complete-transfer-n/', 'content': 'Real Madrid have confirmed the signing of France striker Kylian Mbappé, who is to join the LaLiga club on a five-year contract. ... Madrid to confirm Mbappé deal at "7:15pm at the very latest"'}]Thought:The observation shows a news article reporting Kylian Mbappé's signing with Real Madrid.  However, I need to verify this information and check if there have been any signings since then.  I'll use a different search query to be more comprehensive.

Ac

En general debería usar Wikipedia para información especifica que suele estar contenida en wikipedia, mientras que en cualquier otro caso más general debería usar Tavily. Que tipo de información debería buscar en wikipedia? Cuando se le pregunta el nombre de una persona, una entidad economica, etc. Es posible que incluso en esos casos se use la de Tavily, pero ese sería el criterio.

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

In [63]:
from langchain.tools import tool

@tool
def call_rag(question: str) -> str:
    """Call the RAG model to answer a question"""
    return rag_chain.invoke(question)

@tool
def call_react(input: str) -> str:
    """Call the ReAct model to answer a question"""
    return agent_executor.invoke({"input": input})["output"]

#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

In [ ]:
supervisor_prompt = PromptTemplate.from_template(
    """
    Eres un agente supervisor de preguntas.
    Tu rol es decidir que acción tomar para poder contestar la pregunta de la mejor manera posible.
    - 'llm': Cuando tengas que usar un sistema RAG para extraer información sobre LLMs.
    - 'general': Cuando la pregunta sea relacionada a cosas que puedan estar en internet o personas o entidades que conoce wikipedia.
    - 'otro': Todo aquella pregunta que no esté contenida en las categorías anteriores.

    No respondas con más de una palabra y no incluyas.

    
    {question}
    

    Categoría:"""
)

supervisor_chain = (
    supervisor_prompt
    | llm
    | StrOutputParser()
)

supervisor_chain.invoke({"question": "qué es BERT?"})

'llm\n'

In [ ]:
redirect_prompt = PromptTemplate.from_template(
    """
    Eres un asistente experto en el redireccionamiento de preguntas de usuarios.
    Vas a recibir una pregunta del usuario, tu único rol es indicar que no puedes responder su pregunta y redireccionar al usuario
    para que te pregunte sobre papers relacionados a LLMs, o sobre cualquier cosa que pueda estar en internet o personas o 
    entidades que conoce wikipedia.

    Recuerda ser amable y cordial en tu respuesta.

    Pregunta: {question}
    Respuesta cordial:"""
)

redirect_chain = (
    redirect_prompt
    | llm
    | StrOutputParser()
)

redirect_chain.invoke({"question": "qué estás pensando?"})

NameError: name 'PromptTemplate' is not defined

In [ ]:
def route_question(question):
  '''
  Recibe una pregunta de usuario.
  Rutea la pregunta al agente respectivo y responde de manera acorde.
  '''

  topic = supervisor_chain.invoke({"question": question}) # enrutamiento

  if "llm" in topic:
      return call_rag(question)
  elif "general" in topic:
      return call_react(question)
  else:
      return redirect_chain.invoke({"question": question})

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

In [67]:
print(route_question("Qué es un Transformer?"))

Un Transformer es un modelo de transducción que utiliza únicamente auto-atención para calcular representaciones de su entrada y salida, sin usar RNNs alineados con la secuencia o convoluciones.  Su arquitectura se compone de un codificador y un decodificador, ambos formados por una pila de N=6 capas idénticas. Cada capa tiene dos subcapas: un mecanismo de auto-atención multi-cabeza y una capa totalmente conectada puntual.  Tanto el codificador como el decodificador utilizan codificaciones posicionales.  El modelo base utiliza una tasa de abandono (dropout) de Pdrop = 0.1 y durante el entrenamiento se emplea suavizado de etiquetas con un valor de ϵls = 0.1.



In [68]:
print(route_question("Según Chain of Thought, ¿qué técnica se puede usar para resolver un problema complejo con un LLM?"))

Según la información proporcionada, la técnica Chain of Thought (pensamiento en cadena) se puede usar para resolver problemas complejos con un LLM.  Esta técnica permite descomponer problemas de múltiples pasos en subproblemas más pequeños y manejables,  imitando un proceso de pensamiento paso a paso para llegar a la respuesta.  Además, es robusta a diferentes órdenes de ejemplos y a un número variable de ellos.



In [74]:
print(route_question("¿A qué se dedica Tailor Swift?"))



> Entering new AgentExecutor chain...
Thought: To answer what Taylor Swift does, I need to find information about her profession.  Wikipedia is a good resource for this.

Action: wikipedia
Action Input: Taylor Swift
Page: Taylor Swift
Summary: Taylor Alison Swift (West Reading, Pensilvania; 13 de diciembre de 1989)Thought: The Wikipedia page gives biographical information, but I need to extract her profession.  The summary doesn't explicitly state it, so I'll look further into the page content.

Action: wikipedia
Action Input: Taylor Swift
Page: Taylor Swift
Summary: Taylor Alison Swift (West Reading, Pensilvania; 13 de diciembre de 1989)Thought: The Wikipedia page should contain information about her career. I'll look for sections like "Career" or "Profession".

Action: wikipedia
Action Input: Taylor Swift
Page: Taylor Swift
Summary: Taylor Alison Swift (West Reading, Pensilvania; 13 de diciembre de 1989)Thought: I need to actually *read* the Wikipedia page content this time, not ju

In [70]:
print(route_question("Quién fue el último fichaje del Real Madrid?"))



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find information about the most recent player signing for Real Madrid.  A quick search on Wikipedia might suffice, but a more up-to-date source like a search engine might be better to ensure I have the most recent information.

Action: tavily_search_results_json
Action Input: "Real Madrid latest signing"
[{'url': 'https://en.as.com/soccer/possible-mbappe-signing-with-real-madrid-live-updates-online-as-french-striker-set-to-complete-transfer-n/', 'content': 'Real Madrid have confirmed the signing of France striker Kylian Mbappé, who is to join the LaLiga club on a five-year contract. ... Madrid to confirm Mbappé deal at "7:15pm at the very latest"'}]Thought: The observation shows a news article reporting Kylian Mbappé's signing with Real Madrid.  However, I need to verify this information and check if there have been any signings since then.  I'll use a different search query to be more comprehensive.

A

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

`escriba su respuesta acá`

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [71]:
%pip install --upgrade --quiet gradio

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.1.1 requires langchain-core<0.3,>=0.1.40, but you have langchain-core 0.3.19 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [72]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )

KeyboardInterrupt: 